**Описание ноутбука**

Данный ноутбук представляет собой инструмент для автоматического сбора, очистки и первичной обработки данных о продаже квартир с сайта объявлений недвижимости. В рамках работы осуществляется:

Скачивание web-страниц с предложениями о продаже квартир;

Извлечение ключевой информации о каждой квартире (цена, адрес, район, параметры объекта, описание и т.д.);

Очистка текстовых данных от HTML-тегов, спецсимволов и эмодзи для получения удобочитаемого результата;

Обработка пропущенных или отсутствующих данных;

Сохранение итоговой таблицы в удобном формате для дальнейшего анализа.

Комментарии к коду написаны нейросетью.
Выполнил:
**Студент 121-ПМо Котов Артем Сергеевич**

In [34]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import files
import re

Тут **жостко** проводим сбор всех страниц с объявлениями о продаже квартир на сайте http://magnitogorsk-citystar.ru.
Мы **долго** и тщательно обходим нужный диапазон страниц по номерам, загружаем их содержимое и сразу превращаем каждую страницу в объект BeautifulSoup для КРАСАТЫ.
Полученные объекты сохраняем в список, чтобы потом извлекать с них вкусненькие данные.

In [8]:
# Собираем все страницы с объявлениями о квартирах
pages = []
for i in range(1, 22):
    url = BeautifulSoup(requests.get(
        'http://magnitogorsk-citystar.ru/realty/prodazha-kvartir/?p=' + str(i)
    ).text, 'lxml')
    pages.append(url)


Тут мы для каждой сохранённой ранее страницы ищем таблицу с объявлениями о квартирах и начинаем по строкам вылавливать (рыбку) отдельные объекты.

Для каждой строки, где есть нужное число столбцов, мы достаем уникальный ID квартиры и ссылку на её объявление.
Все найденные id и ссылки складываются в отдельные списки, чтобы дальше смачно их использовать.
Если вдруг на странице таблица не нашлась, мы с позором уходим рыскать дальше :(

In [17]:
all_links_list = []
all_ids_list = []
for page in pages:
    # Находим таблицу с квартирами на странице
    table = page.find('div', {'class': 'wrapper'})
    if table is None:
        continue
    for tr in table.find_all('tr'):
        if len(tr) != 13:
            continue
        # Считываем id квартиры и ссылку на объявление
        id = int(tr.find_all('td', recursive=False)[0].input['value'])
        link = tr.find_all('td', recursive=False)[4].a['href']
        all_ids_list.append(id)
        all_links_list.append(link)

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>


Создаем наш сладенький датафрейм, чтобы наполнить его наивкуснейшей начинкой.

In [18]:
columns = ['Ссылка', 'Цена', 'Район', 'Адрес', 'Этаж',
           'Высота потолка', 'Планировка', 'Площадь',
           'Состояние квартиры', 'Кол-во комнат', 'Описание']
data = pd.DataFrame(index=all_ids_list, columns=columns)
data['Ссылка'] = all_links_list



Здеся мы перебираем все собранные ссылки на квартиры, открываем каждую страницу и извлекаем из объявления начинку объекта.
Для каждой квартиры собираются такие поля: адрес, цена, метраж, этаж, состояние и при наличии текстовое описание.
Информация сначала раскладывается по ключам и значениям, а затем аккуратно добавляется в соответствующие ячейки датафреймы.
В итоге у каждой строки с квартирой постепенно заполняются все внутренности.

In [19]:
# По каждой квартире переходим на её страницу и забираем начиночку
for i in range(len(all_ids_list)):
    url = 'http://magnitogorsk-citystar.ru' + all_links_list[i]
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    div = soup.find('div', 'adv-main-data')

    # Получаем список полей и их значения
    keys = [t.get_text(strip=True) for t in div.find_all('td', 'field-title')]
    values = [t.get_text(strip=True).replace('\xa0', '') for t in div.find_all('td', 'field')]
    flat_info = dict(zip(keys, values))

    # Если есть поле "Описание", то добавляем и его
    note = div.find('td', 'note')
    if note:
        flat_info['Описание'] = note.get_text(strip=True)
    # Сохраняем собранную информацию в нашу датафреймик
    for key in flat_info:
        if key in data.columns:
            data.at[all_ids_list[i], key] = flat_info[key]



Тут мы вычищаем грязный и невкусный столбец "Описание" от грязи и ошметков HTML-тегов.
Шаблоны для смайликов были нагло украдены и приватизированны из интернета.
Также все ячейки без данных заменяются фразой "Данные отсутсвуют", ибо слово "NaN" некрасивое. А так же мы ненавидим ипотеки, поэтому эта фраза была уничтожена.


In [41]:
clean_patterns = [
    (r'<br\s*/?>', '\n'),               # <br> и <br /> замена на перенос
    (r'</?p>', '\n'),                   # <p> и </p> замена на перенос
    (r'</?(b|i|u|strong|em|span|font)>', ''),  # удаляем часто встречающиеся теги форматирования
    (r'<ul>|</ul>', ''),                # списки - удаляем <ul>
    (r'<li>', '\n- '),                  # <li> превращаем в маркер списка
    (r'</li>', ''),                     # </li> просто удаляем
    (r'&bull;|&#8226;|&middot;', '•'),  # спецсимволы: точка/маркер
    (r'&mdash;|—', '—'),                # длинное тире
    (r'&laquo;', '«'),                  # кавычки
    (r'&raquo;', '»'),
    (r'&quot;', '»'),
    (r'&nbsp;|\xa0', ' '),              # неразрывный пробел
    (r'<.*?>', ''),                     # любые остальные HTML-теги
    # УНИВЕРСАЛЬНЫЙ ПАТТЕРН ЭМОДЗИ:
    (r'[\U0001F600-\U0001F64F'  # emoticons
     r'\U0001F300-\U0001F5FF'  # symbols & pictographs
     r'\U0001F680-\U0001F6FF'  # transport & map
     r'\U0001F1E0-\U0001F1FF'  # flags
     r'\U00002700-\U000027BF'  # dingbats
     r'\U000024C2-\U0001F251'  # enclosed characters
     r'\U0001F900-\U0001F9FF'  # supplemental pictographs
     r'\U0001FA70-\U0001FAFF'  # more symbols
     r'\U00002600-\U000026FF'  # misc symbols
     r'\U00002300-\U000023FF'  # misc technical
     r'\U000025A0-\U000025FF'  # geometric shapes
     r']+', ''),
    (r'\n{2,}', '\n'),                  # несколько переносов подряд меняем на один
    (r'^\s+|\s+$', ''),                 # удаляем пробелы в начале и конце строки
]
data = data.fillna('Данные отсутсвуют')
for pattern, repl in clean_patterns:
    data['Описание'] = data['Описание'].str.replace(pattern, repl, regex=True)
    data['Цена'] = data['Цена'].str.replace('Подать заявку на ипотеку', '', regex=True)

Подсматриваем за датафреймом, все ли красиво, а то мало ли...

In [42]:
data

,Ссылка,Цена,Район,Адрес,Этаж,Высота потолка,Планировка,Площадь,Состояние квартиры,Кол-во комнат,Описание
18634318,/realty/prodazha-kvartir/prodam-kvartiru-pravo...,5900000р.(97039р./м2),Правобережный,"Им. газеты ""Правда"", 65/2",1/6,"2,5 м.",хабаровский вариант,"общая60,8 м2,жилая38 м2,кухни10,2 м2",Идеальное,Двухкомнатная,Пpодам 2х кoмнaтную кв. в кирпичном доме на пе...
23524837,/realty/prodazha-kvartir/prodam-kvartiru-pravo...,8750000р.(82160р./м2),Правобережный,"Сталеваров, 17/1",2/10,"2,5 м.",нестандартная,"общая106,5 м2,жилая70 м2,кухни16 м2",Хорошее,Трехкомнатная,Продам трёхкомнатную квартиру в ЖК Ладья. Квар...
23515912,/realty/prodazha-kvartir/prodam-kvartiru-23515...,2700000р.(87948р./м2),Данные отсутсвуют,"Ленина пр-т, 150",5/9,Данные отсутсвуют,Данные отсутсвуют,"общая30,7 м2,жилая16 м2,кухни7 м2",Данные отсутсвуют,Однокомнатная,id:46603. \nПредлагается уютная однокомнатная ...
23515911,/realty/prodazha-kvartir/prodam-kvartiru-23515...,3500000р.(80092р./м2),Данные отсутсвуют,"Советский переулок, 2",3/5,Данные отсутсвуют,Данные отсутсвуют,"общая43,7 м2,жилая30 м2,кухни5,5 м2",Данные отсутсвуют,Двухкомнатная,id:46589. \nЭксклюзивный вариант - двухкомнатн...
23515908,/realty/prodazha-kvartir/prodam-kvartiru-23515...,6500000р.(78313р./м2),Данные отсутсвуют,"Советская, 215",4/9,Данные отсутсвуют,Данные отсутсвуют,"общая83 м2,жилая53 м2,кухни9 м2",Данные отсутсвуют,Четырехкомнатная,id:46569. \n4х комнатная квартира улучшенной п...
...,...,...,...,...,...,...,...,...,...,...,...
16262155,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,5500000р.(84615р./м2),Орджоникидзевский,"Жукова, 19/1",1/10,Данные отсутсвуют,Данные отсутсвуют,"общая65 м2,жилая43 м2,кухни12 м2",Данные отсутсвуют,Двухкомнатная,"В продаже уютная 2к квартира, в одном из лучши..."
15406262,/realty/prodazha-kvartir/prodam-kvartiru-ordzh...,8000000р.(76190р./м2),Орджоникидзевский,"Карла Маркса, 185",4/9,Данные отсутсвуют,Данные отсутсвуют,"общая105 м2,жилая83 м2,кухни10 м2",Данные отсутсвуют,Четырехкомнатная,Предлагаю Вашему вниманию четырёхкомнатную ква...
16022577,/realty/prodazha-kvartir/prodam-kvartiru-lenin...,5560000р.(76374р./м2),Ленинский,"Строителей, 57",6/6,Данные отсутсвуют,Данные отсутсвуют,"общая72,8 м2,жилая44 м2,кухни8 м2",Данные отсутсвуют,Трехкомнатная,Для ценителей Ленинского района в продаже уютн...
15761115,/realty/prodazha-kvartir/prodam-kvartiru-15761...,1950000р.(108333р./м2),Данные отсутсвуют,"Курортная, 19",2/2,Данные отсутсвуют,Данные отсутсвуют,"общая18 м2,жилая15 м2,кухни0 м2",Данные отсутсвуют,Однокомнатная,"В эксклюзивной продаже апартаменты в ЖК""КАРАТА..."


Жесточайше сохраняем датафрейм в файл формата csv и xlsx, чтобы вкусненько на него поглядеть.

In [40]:
data.to_csv('magnitogorsk_parsed.csv')
data.to_excel('magnitogorsk_parsed.xlsx')
files.download('magnitogorsk_parsed.csv')
files.download('magnitogorsk_parsed.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>